In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import os
import tensorflow as tf
import torchvision.transforms as transforms
import torch
import torch.nn as nn
from models.segnet import SegNet
num_classes  = 7
rgb_channels = 3
model = SegNet( num_classes, 
                n_init_features = rgb_channels,
                )

print(model)

SegNet
 Num class: 7 
 Input chanels: 3



In [3]:
from dataloaders.FREIBRG import MFORESTDataset
from dataloaders.FREIBRG import  conv_mask_to_img_np
root = "C:/Users/kunha/Desktop/src/"  # path to the root directory of the dataset 

test_loader = MFORESTDataset( root=root,
                        set='test',
                        rgb_dir = 'rgb',
                        mask_dir = 'GT_color',
                        dsm_dir = 'depth_gray',
                        num_classes = 7 

                        )
train_loader = MFORESTDataset( root=root,
                        set='train',
                        rgb_dir = 'rgb',
                        mask_dir = 'GT_color',
                        dsm_dir = 'depth_gray',
                        num_classes = 7 

                        )
print(train_loader)


MFORESTDataset
 Loaded 230 Files
 Target size: [240, 240]
 Loading mode: DISK


In [4]:
from torch.utils.data import DataLoader
batch_size = 3
train_dataloader = DataLoader(train_loader, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_loader, batch_size=batch_size, shuffle=True)
print("The number of images in a training set is: ", len(train_dataloader)*batch_size)
print(len(train_dataloader)*batch_size)
print(len(test_dataloader)*batch_size)

The number of images in a training set is:  231
231
138


In [4]:
TrainingSIZE =int(len(train_dataloader) *0.8)
ValidationSIZE =int(len(train_dataloader) - TrainingSIZE)
training_data, validation_data = torch.utils.data.random_split(train_dataloader, [TrainingSIZE, ValidationSIZE])
print('Training set has {} instances'.format(len(training_data)))
print('Validation set has {} instances'.format(len(validation_data)))

Training set has 92 instances
Validation set has 23 instances


In [5]:
from torch.optim import Adam
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

In [ ]:
from torch.autograd import Variable

# Function to save the model
def saveModel():
    path = "./Modelo1.pth"
    torch.save(model.state_dict(), path)

# Function to test the model with the test dataset and print the accuracy for the test images
def testAccuracy():
    
    model.eval()
    accuracy = 0
    total = 0
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    with torch.no_grad():
        for data in test_dataloader:
            rgb,dsm,mask,id = data
            rgb, mask = rgb.to(device), mask.to(device)
            #rgb, mask = rgb.cuda(), mask.cuda()
            # run the model on the test set to predict labels
            outputs = model(rgb)
            outputsnp = outputs.cpu()
            outputsnp = np.asarray(outputs)
            outputst = tf.convert_to_tensor(outputsnp)
            #outputs  = torch.stack(outputs, dim=0)
            # the label with the highest energy will be our prediction
            _, predicted = torch.max(outputst, 1)
            total += mask.size(0)
            accuracy += (predicted == mask).sum().item()
    
    # compute the accuracy over all test images
    accuracy = (100 * accuracy / total)
    return(accuracy)


# Training function. We simply have to loop over our data iterator and feed the inputs to the network and optimize.
def train(num_epochs):
    best_accuracy = 0

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #device = torch.device("cpu")

    print("The model will be running on", device, "device")
    
    # Convert model parameters and buffers to CPU or Cuda
    model.to(device)

    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0
        running_acc = 0

        for i, data in enumerate(train_dataloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            rgb,dsm,mask,id = data
            #rgb.to(device)
            #rgb, mask = rgb.cuda(), mask.cuda()
            rgb, mask = rgb.to(device), mask.to(device)
            
            #mask = torch.argmax(mask,axis=1,keepdim=True)
            # zero the parameter gradients
            optimizer.zero_grad()
        
            # predict classes using images from the training set
            outputs,_ = model(rgb)
            #outputs = torch.argmax(outputs,axis=1,keepdim=True)

            # compute the loss based on model output and real labels
            loss = loss_fn(outputs, mask)
            # backpropagate the loss
            loss.backward()
            # adjust parameters based on the calculated gradients
            optimizer.step()

            # Let's print statistics for every 1,000 images
            running_loss += loss.item()     # extract the loss value
            if i % 1000 == 999:    
                # print every 1000 (twice per epoch) 
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 1000))
                # zero the loss
                running_loss = 0

        # Compute and print the average accuracy fo this epoch when tested over all 10000 test images
        accuracy = testAccuracy()
        print('For epoch', epoch+1,'the test accuracy over the whole test set is %d %%' % (accuracy))
        
        # we want to save the model if the accuracy is the best
        if accuracy > best_accuracy:
            saveModel()
            best_accuracy = accuracy

In [6]:
import gc 
gc.collect()
torch.cuda.empty_cache()

In [8]:
num_epochs=8

# keeping-track-of-losses 
train_losses = []
valid_losses = []

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

print("The model will be running on", device, "device")
    
# Convert model parameters and buffers to CPU or Cuda
model.to(device)
best_accuracy = 0
for epoch in range(1, num_epochs + 1):
    # keep-track-of-training-and-validation-loss
    train_loss = 0.0
    #valid_loss = 0.0
    
    # training-the-model
    model.train()
    for data in train_dataloader:
        rgb,dsm,mask,id = data
        rgb, mask = rgb.to(device), mask.to(device)

        
        # clear-the-gradients-of-all-optimized-variables
        optimizer.zero_grad()
        # forward-pass: compute-predicted-outputs-by-passing-inputs-to-the-model
        output,_ = model(rgb)
        # calculate-the-batch-loss
        loss = loss_fn(output, mask)
        # backward-pass: compute-gradient-of-the-loss-wrt-model-parameters
        loss.backward()
        # perform-a-ingle-optimization-step (parameter-update)
        optimizer.step()
        # update-training-loss
        train_loss += loss.item()
        
    """
    # validate-the-model
    model.eval()
    #accuracy = 0
    total = 0
    for data in test_dataloader:
        rgb,dsm,mask,id = data
        rgb, mask = rgb.to(device), mask.to(device)
        
        output,_ = model(rgb)
        #_, predicted = torch.max(output, 1)
        #total += mask.size(0)
        #accuracy += (predicted == mask).sum().item()
        loss = loss_fn(output, mask)
        
        # update-average-validation-loss 
        valid_loss += loss.item()
    """
    # calculate-average-losses
    train_loss = train_loss/len(train_dataloader.sampler)
    #valid_loss = valid_loss/len(test_dataloader.sampler)
    train_losses.append(train_loss)
    #valid_losses.append(valid_loss)
        
    # print-training/validation-statistics 
    #print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
     #   epoch, train_loss, valid_loss))

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))

    #print('For epoch', epoch+1,'the test accuracy over the whole test set is %d %%' % (accuracy))
            
    #if accuracy > best_accuracy:
     #   saveModel()
     #   best_accuracy = accuracy

The model will be running on cuda:0 device
Epoch: 1 	Training Loss: 0.290555
Epoch: 2 	Training Loss: 0.180146
Epoch: 3 	Training Loss: 0.149599
Epoch: 4 	Training Loss: 0.144095
Epoch: 5 	Training Loss: 0.130931
Epoch: 6 	Training Loss: 0.129230
Epoch: 7 	Training Loss: 0.121925
Epoch: 8 	Training Loss: 0.125693


In [9]:
torch.save(model.state_dict(), 'checkpoints/Model_8epochs.pth')

In [8]:
# test-the-model
#classes = ('void','Road','Grass','Vegetation','Tree','Sky','Obstacle')
#correct_pred = {classname: 0 for classname in classes}
#total_pred = {classname: 0 for classname in classes}
model.eval()  # it-disables-dropout
with torch.no_grad():
    correct = 0
    total = 0
    for data in test_dataloader:
        rgb,dsm,mask,id = data
        rgb = rgb.to(device)
        mask = mask.to(device)
        outputs,_ = model(rgb)
        #outputs = torch.tensor(outputs)
        #outputs  = torch.stack(outputs, dim=0)
        #predicted,_ = torch.max(outputs.data,dim=1,keepdim=True)
        _, predicted = torch.max(outputs, 1,keepdim=True)
        total += mask.size(0)
        correct += (predicted == mask).sum().item()
          
    print('Test Accuracy of the model: {} %'.format(100 * correct / total))


# Save 
#path = os.path.join('checkpoints') import os
torch.save(model.state_dict(), 'checkpoints/Modelo1.pth')

Test Accuracy of the model: 579613.9705882353 %


In [10]:
def check_accuracy(loader, model):
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    model.eval()

    with torch.no_grad():
        for data in loader:
            rgb,dsm,mask,id = data
            rgb,mask = rgb.to(device), mask.to(device)
            softmax = nn.Softmax(dim=1)
            outputs,_ = model(rgb)
            preds = torch.argmax(softmax(outputs),axis=1,keepdim=True)
            num_correct += (preds == mask).sum()
            num_pixels += torch.numel(preds)
            dice_score += (2 * (preds * mask).sum()) / ((preds + mask).sum() + 1e-8)

    print(f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}")
    print(f"Dice score: {dice_score/len(loader)}")
    model.train()

In [13]:
check_accuracy(train_dataloader, model)

Got 1329848/13248000 with acc 10.04
Dice score: 0.2767717242240906


In [12]:
check_accuracy(test_dataloader, model)

Got 761294/7833600 with acc 9.72
Dice score: 0.2768886387348175
